# 第 5 单元：ML-Agents 简介



<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit7/thumbnail.png" alt="Thumbnail"/>

在这个 notebook 中，你将学习 ML-Agents 并训练两个智能体。

- 第一个将学习**向生成的目标发射雪球**。
- 第二个需要按下一个按钮来生成一个金字塔，然后导航到金字塔，将其推倒，**并移动到顶部的金砖**。为此，它需要探索其环境，我们将使用一种称为“好奇心”的技术。

之后，你将能够**直接在浏览器中观看你的智能体玩游戏**。

有关认证过程的更多信息，请查看此部分 👉 https://huggingface.co/deep-rl-course/en/unit0/introduction#certification-process

⬇️ 这是**你在本单元结束时将实现**的示例。⬇️


<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit7/pyramids.gif" alt="Pyramids"/>

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit7/snowballtarget.gif" alt="SnowballTarget"/>

### 🎮 环境：

- [金字塔 (Pyramids)](https://github.com/Unity-Technologies/ml-agents/blob/main/docs/Learning-Environment-Examples.md#pyramids)
- 雪球目标 (SnowballTarget)

### 📚 RL-库：

- [ML-Agents](https://github.com/Unity-Technologies/ml-agents)


我们一直在努力改进我们的教程，所以**如果你在这个 notebook 中发现任何问题**，请[在 GitHub 仓库中提出一个 issue](https://github.com/huggingface/deep-rl-class/issues)。

## 这个 notebook 的目标 🏆

在本 notebook 结束时，你将：

- 了解环境库 **ML-Agents** 的工作原理。
- 能够**在 Unity 环境中训练智能体**。


## 这个 notebook 来自深度强化学习课程
<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/deep-rl-course-illustration.jpg" alt="Deep RL Course illustration"/>

在这门免费课程中，你将：

- 📖 **理论与实践相结合**学习深度强化学习。
- 🧑‍💻 学习**使用著名的深度强化学习库**，如 Stable Baselines3, RL Baselines3 Zoo, CleanRL 和 Sample Factory 2.0。
- 🤖 在**独特的环境中训练智能体**

更多内容请查看 📚 教学大纲 👉 https://huggingface.co/deep-rl-course/communication/publishing-schedule

别忘了**<a href="http://eepurl.com/ic5ZUD">注册课程</a>**（我们收集你的电子邮件是为了**在每个单元发布时向你发送链接，并为你提供有关挑战和更新的信息）。**


保持联系的最佳方式是加入我们的 discord 服务器，与社区和我们交流 👉🏻 https://discord.gg/ydHrjt3WP5

## 先决条件 🏗️
在深入学习这个 notebook 之前，你需要：

🔲 📚 **通过阅读第 5 单元来学习 [什么是 ML-Agents 及其工作原理](https://huggingface.co/deep-rl-course/unit5/introduction)** 🤗  

# 让我们开始训练我们的智能体吧 🚀

**要为认证过程验证此实践，你只需将训练好的模型推送到 Hub**。这个没有要达到的结果来验证。但如果你想获得好的结果，你可以尝试达到：

- 对于 `Pyramids`：平均奖励 = 1.75
- 对于 `SnowballTarget`：平均奖励 = 15 或在一个回合中击中 30 个目标。


## 设置 GPU 💪
- 为了**加速智能体的训练，我们将使用 GPU**。为此，请转到 `Runtime > Change Runtime type`

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step1.jpg" alt="GPU Step 1">

- `硬件加速器 > GPU`

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step2.jpg" alt="GPU Step 2">

## 克隆仓库 🔽

- 我们需要克隆包含 **ML-Agents** 的仓库。

In [1]:
%%capture
# 克隆仓库（可能需要 3 分钟）
!git clone --depth 1 https://github.com/Unity-Technologies/ml-agents

## 设置虚拟环境 🔽
- 为了让 **ML-Agents** 在 Colab 中成功运行，Colab 的 Python 版本必须满足库的 Python 要求。

- 我们可以在 `setup.py` 文件中的 `python_requires` 参数下检查支持的 Python 版本。这些文件是设置 **ML-Agents** 库所必需的，可以在以下位置找到：
  - `/content/ml-agents/ml-agents/setup.py`
  - `/content/ml-agents/ml-agents-envs/setup.py`

- Colab 当前的 Python 版本（可以使用 `!python --version` 检查）与库的 `python_requires` 参数不匹配，因此安装可能会静默失败，并在稍后执行相同命令时导致如下错误：
  - `/bin/bash: line 1: mlagents-learn: command not found`
  - `/bin/bash: line 1: mlagents-push-to-hf: command not found`

- 为了解决这个问题，我们将创建一个与 **ML-Agents** 库兼容的 Python 版本的虚拟环境。

`注意：` *为了将来的兼容性，请务必检查安装文件中的 `python_requires` 参数，并在 Colab 的 Python 版本不兼容时，在下面的脚本中将你的虚拟环境设置为支持的最高 Python 版本*

In [ ]:
# Colab 当前的 Python 版本（与 ML-Agents 不兼容）
!python --version

In [ ]:
# 安装 virtualenv 并创建一个虚拟环境
!pip install virtualenv
!virtualenv myenv

# 下载 Miniconda 安装脚本
!wget https://repo.anaconda.com/miniconda/Miniconda3-py310_23.11.0-2-Linux-x86_64.sh
# 添加执行权限
!chmod +x Miniconda3-py310_23.11.0-2-Linux-x86_64.sh
# 静默安装到 Colab 的标准目录
!bash ./Miniconda3-py310_23.11.0-2-Linux-x86_64.sh -b -f -p /usr/local
# 接受服务条款，防止后续命令出错
!conda config --set auto_activate_base false
!conda tos accept --override-channels --channel pkgs/main
# 创建名为 mlagents 的新环境，并指定 Python 版本
!conda create -n mlagents python=3.10.12 -y

# 验证一下新环境中的 Python 版本，确保无误
!/usr/local/envs/mlagents/bin/python --version
# 预期的输出应该是： Python 3.10.12

In [ ]:
# 新虚拟环境中的 Python 版本（与 ML-Agents 兼容）
!python --version

## 安装依赖项 🔽

In [ ]:
%%bash
# 激活我们刚刚创建的 conda 环境
source /usr/local/bin/activate mlagents

# 进入 ml-agents 仓库目录
cd ml-agents

echo "--- 正在安装 ml-agents-envs ---"
pip install -e ./ml-agents-envs

echo "--- 正在安装 ml-agents ---"
pip install -e ./ml-agents

echo "--- 依赖安装完成 ---"

## 雪球目标 (SnowballTarget) ⛄

如果你需要复习这个环境的工作原理，请查看此部分 👉
https://huggingface.co/deep-rl-course/unit5/snowball-target

### 下载环境 zip 文件并将其移动到 `./training-envs-executables/linux/`
- 我们的环境可执行文件在一个 zip 文件中。
- 我们需要下载它并将其放置到 `./training-envs-executables/linux/`
- 我们使用 linux 可执行文件，因为我们使用 colab，而 colab 机器的操作系统是 Ubuntu (linux)


In [ ]:
# 这里，我们创建 training-envs-executables 和 linux 目录
!mkdir ./training-envs-executables
!mkdir ./training-envs-executables/linux

我们使用 `wget` 从 https://github.com/huggingface/Snowball-Target 下载了 SnowballTarget.zip 文件

In [ ]:
!wget "https://github.com/huggingface/Snowball-Target/raw/main/SnowballTarget.zip" -O ./training-envs-executables/linux/SnowballTarget.zip

我们解压 executable.zip 文件

In [ ]:
%%capture
!unzip -d ./training-envs-executables/linux/ ./training-envs-executables/linux/SnowballTarget.zip

确保你的文件是可访问的

In [ ]:
!chmod -R 755 ./training-envs-executables/linux/SnowballTarget

### 定义 SnowballTarget 配置文件
- 在 ML-Agents 中，你需要在 `config.yaml` 文件中定义**训练超参数**。

有很多超参数。为了更好地了解它们，你应该查阅[文档](https://github.com/Unity-Technologies/ml-agents/blob/release_20_docs/docs/Training-Configuration-File.md)中对每一个的解释。


所以你需要在 `./content/ml-agents/config/ppo/` 中创建一个 `SnowballTarget.yaml` 配置文件。

我们会在这里给你这个配置的第一个版本（复制并粘贴到你的 `SnowballTarget.yaml` 文件中），**但你应该修改它**。

```
behaviors:
  SnowballTarget:
    trainer_type: ppo
    summary_freq: 10000
    keep_checkpoints: 10
    checkpoint_interval: 50000
    max_steps: 200000
    time_horizon: 64
    threaded: false
    hyperparameters:
      learning_rate: 0.0003
      learning_rate_schedule: linear
      batch_size: 128
      buffer_size: 2048
      beta: 0.005
      epsilon: 0.2
      lambd: 0.95
      num_epoch: 3
    network_settings:
      normalize: false
      hidden_units: 256
      num_layers: 2
      vis_encode_type: simple
    reward_signals:
      extrinsic:
        gamma: 0.99
        strength: 1.0
```

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit7/snowballfight_config1.png" alt="Config SnowballTarget"/>
<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit7/snowballfight_config2.png" alt="Config SnowballTarget"/>

作为实验，你还应该尝试修改一些其他的超参数。Unity 提供了非常[好的文档，在这里解释了每一个超参数](https://github.com/Unity-Technologies/ml-agents/blob/main/docs/Training-Configuration-File.md)。

现在你已经创建了配置文件并了解了大多数超参数的作用，我们准备好训练我们的智能体了 🔥。

### 训练智能体

要训练我们的智能体，我们只需要**启动 mlagents-learn 并选择包含环境的可执行文件**。

我们定义了四个参数：

1. `mlagents-learn <config>`：超参数配置文件的路径。
2. `--env`：环境可执行文件的位置。
3. `--run_id`：你想要为你的训练运行指定的名称。
4. `--no-graphics`：在训练期间不启动可视化。

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit7/mlagentslearn.png" alt="MlAgents learn"/>

训练模型并使用 `--resume` 标志在中断的情况下继续训练。

> 如果以及当你使用 `--resume` 时第一次失败，请尝试再次运行该代码块以绕过错误。



根据你的配置，训练将花费 10 到 35 分钟，去喝杯 ☕️ 吧，你值得拥有 🤗。

In [ ]:
!mlagents-learn ./config/ppo/SnowballTarget.yaml --env=./training-envs-executables/linux/SnowballTarget/SnowballTarget --run-id="SnowballTarget1" --no-graphics

### 将智能体推送到 🤗 Hub

- 现在我们已经训练了我们的智能体，我们**准备好将它推送到 Hub，以便能够在你的浏览器上可视化它的游戏过程🔥。**

为了能够与社区分享你的模型，还需要遵循三个步骤：

1️⃣（如果还没有）创建一个 HF 账户 ➡ https://huggingface.co/join

2️⃣ 登录，然后你需要从 Hugging Face 网站存储你的身份验证令牌。
- 创建一个新令牌（https://huggingface.co/settings/tokens）**并赋予 write 角色**

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/create-token.jpg" alt="Create HF Token">

- 复制令牌
- 运行下面的单元格并粘贴令牌

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

如果你不想使用 Google Colab 或 Jupyter Notebook，你需要改用这个命令：`huggingface-cli login`

然后，我们只需要运行 `mlagents-push-to-hf`。

我们定义了 4 个参数：

1. `--run-id`：训练运行的 ID 名称。
2. `--local-dir`：智能体的保存位置，即 `results/<run_id name>`，所以在我的例子中是 `results/First Training`。
3. `--repo-id`：你想要创建或更新的 Hugging Face 仓库的名称。格式总是 `<你的 huggingface 用户名>/<仓库名>`
如果仓库不存在，**它将被自动创建**
4. `--commit-message`：因为 HF 仓库是 git 仓库，所以你需要定义一个提交信息。

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit7/mlagentspushtohub.png" alt="Push to Hub"/>

例如：

`!mlagents-push-to-hf  --run-id="SnowballTarget1" --local-dir="./results/SnowballTarget1" --repo-id="ThomasSimonini/ppo-SnowballTarget"  --commit-message="First Push"`

In [ ]:
!mlagents-push-to-hf --run-id="SnowballTarget1" --local-dir="./results/SnowballTarget1" --repo-id="ThomasSimonini/ppo-SnowballTarget" --commit-message="First Push"

In [ ]:
!mlagents-push-to-hf  --run-id= # 添加你的运行 ID  --local-dir= # 你的本地目录  --repo-id= # 你的仓库 ID  --commit-message= # 你的提交信息

另外，如果一切顺利，你应该在过程结束时看到这个（但 URL 不同 😆）：



```
Your model is pushed to the hub. You can view your model here: [https://huggingface.co/ThomasSimonini/ppo-SnowballTarget](https://huggingface.co/ThomasSimonini/ppo-SnowballTarget)
```

这是你模型的链接，它包含一个模型卡片，解释了如何使用它，你的 Tensorboard 和你的配置文件。**最棒的是它是一个 git 仓库，这意味着你可以有不同的提交，用新的推送更新你的仓库等等。**

但现在最精彩的部分来了：**能够在线观看你的智能体 👀。**

### 观看你的智能体玩游戏 👀

这一步很简单：

1. 前往这里：https://huggingface.co/spaces/ThomasSimonini/ML-Agents-SnowballTarget

2. 启动游戏并通过点击右下角按钮将其全屏

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit7/snowballtarget_load.png" alt="Snowballtarget load"/>

1. 在步骤 1 中，输入你的用户名（你的用户名区分大小写：例如，我的用户名是 ThomasSimonini，而不是 thomassimonini 或 ThOmasImoNInI）并点击搜索按钮。

2. 在步骤 2 中，选择你的模型仓库。

3. 在步骤 3 中，**选择你想要重放的模型**：
  - 我有多个模型，因为我们每 500000 个时间步保存一个模型。
  - 但因为我想要最新的，我选择 `SnowballTarget.onnx`

👉 一个不错的做法是**尝试用不同步骤的模型来观察智能体的进步。**

不要犹豫，在 discord 的 #rl-i-made-this 频道分享你的智能体获得的最高分 🔥

现在让我们尝试一个更难的环境，叫做 Pyramids...


## 金字塔 (Pyramids) 🏆

### 下载环境 zip 文件并将其移动到 `./training-envs-executables/linux/`
- 我们的环境可执行文件在一个 zip 文件中。
- 我们需要下载它并将其放置到 `./training-envs-executables/linux/`
- 我们使用 linux 可执行文件，因为我们使用 colab，而 colab 机器的操作系统是 Ubuntu (linux)


我们使用 `wget` 从 https://huggingface.co/spaces/unity/ML-Agents-Pyramids/resolve/main/Pyramids.zip 下载了 Pyramids.zip 文件

In [ ]:
!wget "https://huggingface.co/spaces/unity/ML-Agents-Pyramids/resolve/main/Pyramids.zip" -O ./training-envs-executables/linux/Pyramids.zip

我们解压 executable.zip 文件

In [ ]:
%%capture
!unzip -d ./training-envs-executables/linux/ ./training-envs-executables/linux/Pyramids.zip

确保你的文件是可访问的

In [ ]:
!chmod -R 755 ./training-envs-executables/linux/Pyramids/Pyramids

### 修改 PyramidsRND 配置文件
- 与第一个自定义环境不同，**Pyramids 是由 Unity 团队制作的**。
- 所以 PyramidsRND 配置文件已经存在，位于 `./content/ml-agents/config/ppo/PyramidsRND.yaml`
- 你可能会问为什么 PyramidsRND 中有 “RND”。RND 代表*随机网络蒸馏*，这是一种产生好奇心奖励的方法。如果你想了解更多关于这项技术的信息，我们写了一篇文章来解释它：https://medium.com/data-from-the-trenches/curiosity-driven-learning-through-random-network-distillation-488ffd8e5938

对于这次训练，我们将修改一件事：
- 总训练步数超参数太高了，因为我们仅用 100 万个训练步数就可以达到基准（平均奖励 = 1.75）。
👉 为此，我们进入 `config/ppo/PyramidsRND.yaml`，**并将 `max_steps` 修改为 `1000000`。**

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit7/pyramids-config.png" alt="Pyramids config"/>

作为实验，你还应该尝试修改一些其他的超参数，Unity 在[这里提供了非常好的文档来解释它们中的每一个](https://github.com/Unity-Technologies/ml-agents/blob/main/docs/Training-Configuration-File.md)。

我们现在准备好训练我们的智能体了 🔥。

### 训练智能体

根据你的机器，训练将花费 30 到 45 分钟，去喝杯 ☕️ 吧，你值得拥有 🤗。

In [ ]:
!mlagents-learn ./config/ppo/PyramidsRND.yaml --env=./training-envs-executables/linux/Pyramids/Pyramids --run-id="Pyramids Training" --no-graphics

### 将智能体推送到 🤗 Hub

- 现在我们已经训练了我们的智能体，我们**准备好将它推送到 Hub，以便能够在你的浏览器上可视化它的游戏过程🔥。**

In [ ]:
!mlagents-push-to-hf  --run-id= # 添加你的运行 ID  --local-dir= # 你的本地目录  --repo-id= # 你的仓库 ID  --commit-message= # 你的提交信息

### 观看你的智能体玩游戏 👀

👉 https://huggingface.co/spaces/unity/ML-Agents-Pyramids

### 🎁 奖励：为什么不试试在另一个环境中训练呢？
现在你已经知道如何使用 MLAgents 训练智能体了，**为什么不试试另一个环境呢？**

MLAgents 提供了 17 种不同的环境，我们也在构建一些自定义的环境。学习的最佳方式是亲自动手尝试，祝你玩得开心。



![cover](https://miro.medium.com/max/1400/0*xERdThTRRM2k_U9f.png)

你可以在这里找到 Unity 官方环境的完整列表 👉 https://github.com/Unity-Technologies/ml-agents/blob/develop/docs/Learning-Environment-Examples.md

用于可视化你的智能体的演示 👉 https://huggingface.co/unity

目前我们集成了：
- [Worm](https://huggingface.co/spaces/unity/ML-Agents-Worm) 演示，你在其中教**一条虫子爬行**。
- [Walker](https://huggingface.co/spaces/unity/ML-Agents-Walker) 演示，你在其中教一个智能体**走向一个目标**。

今天就到这里。恭喜你完成了本教程！

学习的最佳方式是实践和尝试。为什么不试试另一个环境呢？ML-Agents 有 17 种不同的环境，但你也可以创建自己的环境？查看文档，玩得开心！

我们第 6 单元再见 🔥，

## 持续学习，保持出色 🤗